Following: https://pytorch.org/tutorials/intermediate/torchvision_tutorial.html

In [4]:
import matplotlib.pyplot as plt 
from torchvision.io import read_image

image = read_image("data/PennFudanPed/PNGImages/FudanPed00046.png")
mask = read_image("data/PennFudanPed/PedMasks/FudanPed00046_mask.png")

RuntimeError: isString() INTERNAL ASSERT FAILED at "/Users/ec2-user/runner/_work/_temp/conda_environment_4789825307/lib/python3.11/site-packages/torch/include/ATen/core/ivalue_inl.h":2289, please report a bug to PyTorch. Expected String but got Blob